# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

## Разбейте данные на выборки

Подготовим функцию, при помощи которой разделим имеющийся датасет на три выборки. Стандартная функция предоставляет возможность выполнить разделение только на две выборки, что нам не подходит. 

In [5]:
def three_samples_split(features, target, train_sample_size=0.50, validation_sample_size=0.25):
    from sklearn.model_selection import train_test_split
    initial_data_lenght = len(target)
    train_features, df_features, train_target, df_target = train_test_split(
        features,
        target,
        train_size=train_sample_size,
        random_state=12345
    )
    current_validation_sample_size = (initial_data_lenght * validation_sample_size) / len(df_target)
    validation_features, test_features, validation_target, test_target = train_test_split(
        df_features,
        df_target,
        train_size=current_validation_sample_size,
        random_state=12345
    )
    return train_features, train_target, validation_features, validation_target, test_features, test_target

In [6]:
train_features, train_target, validation_features, validation_target, test_features, test_target = three_samples_split(features, target)

In [7]:
print(train_features.shape)
print(validation_features.shape)
print(test_features.shape)

(1607, 4)
(803, 4)
(804, 4)


На выходе мы получили три выборки заданных размеров.

## Исследуйте модели

In [8]:
#посмотрим на значения целевого признака
target.unique()

array([0, 1])

Мы видим, что целевой признак принимает только два значения: 0 и 1. Это означает, что целевой признак является булевым, то есть категорийным. Следовательно, для его предсказания потребутся решить задачу классификации. Для решения задачи классификации мы можем использовать три алгоритма: 

* Логистическая регрессия
* Дерево решений
* Случайный лес

### Логистическая регрессия

In [9]:
#начнем с логистической регрессии
model = LogisticRegression(random_state=12345, solver='lbfgs')
model = model.fit(train_features, train_target)
result_validation = model.score(validation_features, validation_target)
result_test = model.score(test_features, test_target)

In [10]:
report = classification_report(
    y_true=validation_target,
    y_pred=model.predict(validation_features),
    target_names=['Smart', 'Ultra'])

print(report)

              precision    recall  f1-score   support

       Smart       0.76      0.94      0.84       570
       Ultra       0.66      0.27      0.38       233

    accuracy                           0.75       803
   macro avg       0.71      0.61      0.61       803
weighted avg       0.73      0.75      0.71       803



In [11]:
report = classification_report(
    y_true=test_target,
    y_pred=model.predict(test_features),
    target_names=['Smart', 'Ultra'])

print(report)

              precision    recall  f1-score   support

       Smart       0.77      0.97      0.86       571
       Ultra       0.78      0.27      0.41       233

    accuracy                           0.77       804
   macro avg       0.77      0.62      0.63       804
weighted avg       0.77      0.77      0.73       804



Мы видим, что accuracy полученной модели находится на отметке, близкой к 0.75, что является минимально допустимым значением. Однако назвать прогноз модели на нельзя назвать в полной мере подходящим для поставленных целей - мы видим, что recall на обеих выборках дает не очень хороший результат для целевого класса - всего 27%. Это означает, что модель возвращает очень много ложно-позитивных результатов, то есть ошибок первого рода.

Для бизнеса - это необходимость задействовать дополнительные ресурсы на контакт с пользователями, которые на самом деле не нуждаются в тарифе "Ультра". 

### Дерево решений

In [12]:
model = DecisionTreeClassifier(random_state=12345)

gscv = GridSearchCV(
    model,
    {
        'max_depth': [x for x in range(train_features.shape[1], 21)],
        'criterion': ['gini', 'entropy']
    },
    cv=5,
    return_train_score=False,
    scoring = 'recall'
)

gscv.fit(train_features, train_target)
report = pd.DataFrame(gscv.cv_results_)
report = report[['param_criterion', 'param_max_depth', 'mean_test_score', 'mean_fit_time']]
report.sort_values(by='mean_test_score', ascending=False).head(10)

,param_criterion,param_max_depth,mean_test_score,mean_fit_time
32,entropy,19,0.603041,0.009855
33,entropy,20,0.603036,0.009955
31,entropy,18,0.597261,0.009948
15,gini,19,0.595335,0.009052
29,entropy,16,0.589573,0.009989
16,gini,20,0.589555,0.009370
11,gini,15,0.589543,0.007747
30,entropy,17,0.587603,0.009706
13,gini,17,0.585708,0.008214
27,entropy,14,0.583775,0.009802


In [13]:
decision_tree_classifier = DecisionTreeClassifier(criterion='entropy', max_depth=19, random_state=12345)
decision_tree_classifier.fit(train_features, train_target)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=19,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')

In [14]:
report = classification_report(
    y_true=test_target,
    y_pred=decision_tree_classifier.predict(test_features),
    target_names=['Smart', 'Ultra'])

print(report)

              precision    recall  f1-score   support

       Smart       0.83      0.83      0.83       571
       Ultra       0.58      0.58      0.58       233

    accuracy                           0.76       804
   macro avg       0.71      0.70      0.70       804
weighted avg       0.76      0.76      0.76       804



Наблюдаем более интересные результаты у модели на базе алогритма "Дерево решений":

1. Accuracy выросла на 1 процентный пункт и составляет 76%
2. Наиболее проблемная метрика Recall также выросла до 58%

### Случайный лес

Попробуем другой алгоритм - случайный лес. Для перебора гиперпараметров также воспользуемся функцией, написанной ранее, внеся в нее некоторые модификации. 

В случае случайного леса мы будем использовать разные значения таких параметров как: 

* Критерий 
* Максимальная глубина одного дерева
* Количество деревьев
* Минимальное количество значений для сплита дерева 

Управлять этими параметрами будем, отталкиваясь от значений аргументов, передаваемых нашей функции. 

In [15]:
def random_forest_classifier(train_features,
                             target_train,
                             valid_features,
                             valid_target,
                             n_estimators,
                             max_depth,
                             min_samples_split
                            ):
    best_result = 0
    for criterion in ['gini', 'entropy']:
        print('Critetion', criterion)
        for depth in range(train_features.shape[1], max_depth+1):
            print('Depth =', depth)
            for number in range(100, n_estimators+100, 100):
                for min_samples in range(2, int((len(train_features)*min_samples_split))+1):
                    model = RandomForestClassifier(criterion=criterion, max_depth=depth, n_estimators=number, random_state=12345, min_samples_split=min_samples)
                    model.fit(train_features, target_train)
                    result = model.score(valid_features, valid_target)
                    if result > best_result:
                        best_model = model
                        best_result = result
                        print(f'Best accuracy changed to {best_result:.2%}')
    print(f'Наилучший результат {best_result:.2%}')
    return best_model

In [16]:
random_forest = random_forest_classifier(
    train_features,
    train_target,
    validation_features,
    validation_target,
    1000,
    10,
    0.01)

Critetion gini
Depth = 4
Best accuracy changed to 79.33%
Depth = 5
Best accuracy changed to 79.45%
Depth = 6
Depth = 7
Best accuracy changed to 79.58%
Depth = 8
Best accuracy changed to 79.70%
Depth = 9
Best accuracy changed to 79.95%
Depth = 10
Critetion entropy
Depth = 4
Depth = 5
Depth = 6
Depth = 7
Depth = 8
Depth = 9
Depth = 10
Best accuracy changed to 80.07%
Наилучший результат 80.07%


In [17]:
report = classification_report(
    y_true=test_target,
    y_pred=random_forest.predict(test_features),
    target_names=['Smart', 'Ultra'])

print(report)

              precision    recall  f1-score   support

       Smart       0.84      0.94      0.88       571
       Ultra       0.79      0.55      0.64       233

    accuracy                           0.83       804
   macro avg       0.81      0.74      0.76       804
weighted avg       0.82      0.83      0.82       804



In [18]:
random_forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

Наблюдаем улучшение относительно дерева решений:

1. Accuracy выросла до 83%
2. Recall для целевого признака на тестовой выборке прибавил 15 процентных пунктов и остановился до отметки в 55%. Таким образом, доля ошибок первого рода составила менее половины, что является наилучшим результатом
3. Наилучший результат случайный лес показал при следующих параметрах:
    + Критерий - энтропия
    + Максимальная глубина - 10 деревьев
    + Минимальное количество элементов для сплита - 5
    + Количество деревьев - 300

## (бонус) Проверьте модели на адекватность

Для проверки модели на адекватность воспользуемся DummyClassifier. Не будем передавать ему никакие гиперпараметры кроме random_state. 

In [19]:
model = DummyClassifier(random_state=12345)
model.fit(train_features, train_target)
dummy_prediction = model.predict(test_features)

In [20]:
report = classification_report(
    y_true=test_target,
    y_pred=dummy_prediction,
    target_names=['Smart', 'Ultra'])

print(report)

              precision    recall  f1-score   support

       Smart       0.72      0.67      0.69       571
       Ultra       0.30      0.35      0.32       233

    accuracy                           0.58       804
   macro avg       0.51      0.51      0.51       804
weighted avg       0.60      0.58      0.59       804



Мы видим, что наша обученная модель показывает значительно лучшие результаты по всем метрикам без исключения. 

## Выводы

1. В ходе обучения моделей использованы три различных алгоритма:
    + Логистическая регрессия
    + Дерево решений
    + Случайный лес
2. Наихудший результат по целевой метрике на тестовой выборке показала логистическая регрессия. Accuracy - 77%.
3. Наилучший результат демонстрирует случайный лес. Accuracy - 83%. 
4. Требуемое значение целевой метрики (Accuracy) достигнуто во всех 3 случаях
5. В ходе подбора наилучшей комбинации гиперпараметров случайного леса проводились эксперименты с 4 из них:
    + Критерий
    + Максимальная глубина
    + Минимальное количество элементов для сплита
    + Количество деревьев
6. Помимо наилучшей целевой метрики, случайный лес также показывает улучшение Recall: 27%, 43%, 55% соответственно у алгоритмов, приведенных в пункте 1. Таким образом, помимо наиболее высокой Accuracy случайный лес также лучше справляется с определением целевого класса объектов.